---
# **CHINESE**
---

### **STEP 1: Downloading and installing the dependencies:**
>>**English PyMUSAS RuleBasedTagger**

>>**English spaCy pipeline**

>>**colorama:** *Colorama is a Python module that displays colored output in consoles.*
---
First download both the [Chinese PyMUSAS RuleBasedTagger](https://github.com/UCREL/pymusas-models/releases/tag/cmn_dual_upos2usas_contextual-0.3.3) SpaCy component and the small [Chinese SpaCy model](https://spacy.io/models/zh):


In [ ]:
!pip install https://github.com/UCREL/pymusas-models/releases/download/cmn_dual_upos2usas_contextual-0.3.3/cmn_dual_upos2usas_contextual-0.3.3-py3-none-any.whl
!python -m spacy download zh_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 3.1.1 requires click>=8.1.3, but you have click 8.0.4 which is incompatible.
dask 2024.12.1 requires click>=8.1, but you have click 8.0.4 which is incompatible.
dask-cuda 25.2.0 requires click>=8.1, but you have click 8.0.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 42.5 MB/s eta 0:00:00
✔ Download and installation suc

In [ ]:
!pip install colorama

---
### **STEP 2: Creating the tagger**

---



In [ ]:
import spacy

# We exclude the following components as we do not need them.
nlp = spacy.load('zh_core_web_sm', exclude=['parser', 'ner'])
# Load the Chinese PyMUSAS rule-based tagger in a separate spaCy pipeline
chinese_tagger_pipeline = spacy.load('cmn_dual_upos2usas_contextual')
# Adds the Chinese PyMUSAS rule-based tagger to the main spaCy pipeline
nlp.add_pipe('pymusas_rule_based_tagger', source=chinese_tagger_pipeline)

---

### **STEP 3: Importing the libraries**

---



In [ ]:
import re
import pandas as pd
import numpy as np
from colorama import Fore, Back, Style

import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


---
### **STEP 4: Tagging text:** *tag_text function*

>>**input:** piece of text

>>**output:** python dictionary where *keys* represent the index of a sentence in the input text and *values* represent the usas semantic tags for that sentence

---



In [ ]:
import re

# Defining the tagging function
def tag_text(text):
  usas_eval_tags = {}
  usas_list = {}
  sentences = re.split(r'(。|！|\!|？|\?)', text)
  for i in range(len(sentences)):
    if(sentences[i]):
      print(Fore.GREEN + Style.BRIGHT + "------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
      print(Fore.CYAN + Style.BRIGHT + f"Sentence {i}:", sentences[i])
      print(Fore.CYAN + Style.BRIGHT + "------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
      sent = nlp(sentences[i])
      for token in sent:
        tags = token._.pymusas_tags
        print(Fore.CYAN + f'{token.text}:' + Fore.GREEN, tags)
        for tag in tags:
          usas_list[token.text] = token._.pymusas_tags
      usas_eval_tags[i] = usas_list
  return usas_eval_tags

In [ ]:
# Input text 1
text_1 = """ 你好，同志们，我想和你们分享我在手术后对抗癌症的经历，我又踢进一球，将比分改写为3比0。希望这个故事能够激励某个人。

我在2020年5月被确诊为结肠癌，当时因为大出血而入院。一眨眼之间，医生已经为我切除了30厘米的结肠。恢复过程虽然辛苦但还能应付。好在虽然肿瘤很大，淋巴结却是清晰的，我被诊断为二期癌症。按照医生说的，我已经完全清除癌症，算是比较幸运的人之一，身后没有癌症，只剩一段可在酒吧里说说的好故事。那时，比分是我1-0癌症。

然而，后来我痛苦地发现，癌症也可以经血液转移。2021年1月，我的例行检查显示出了不好的消息，我的肝脏里有好几处病灶，当外科医生告诉我肝转移无法手术时，我的世界崩塌了。我现在是四期结肠癌，伴有肝转移，无法手术。而我还有两个小儿子，一个3岁，一个5岁，我无法让他们没了爸爸。

这一切都发生在新冠封锁时期，家人和孩子因为医院的限制，没办法随时陪伴我。

然而，我并不想放弃，我把它看作是我需要面对的新挑战。一开始胜算不大，但经过长达一年半的化疗和放疗，我身上的癌症彻底被击退，我甚至不需要接受肝脏手术。现在比分是2-0，我和家人都在默默祈祷癌症永远不会再回来。

它确实没有再回来，但完全不同的一场挑战又向我走来。医生在常规检查时怀疑我甲状腺出了问题，经过额外检查和穿刺活检，我被确诊为甲状腺癌。我当时只说了一句：“呃，这次算小菜一碟。”

昨天，我进行了甲状腺全切手术，肿瘤已经有2厘米大。

现在是3-0。我不知道接下来会面对什么，但无论发生什么，我都会和这个混蛋斗争到底。现在我每6个月都会进行预防性的化疗，并继续享受和孩子们共度的时光。

所以，同志，面对你的挑战吧。不管生活有多么艰难，你也有力量和乐观坚持下去。昂起头，继续进球！"""

In [ ]:
# Tagging input text 1
text1_tags = tag_text(text_1)

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sentence 0:  你好，同志们，我想和你们分享我在手术后对抗癌症的经历，我又踢进一球，将比分改写为3比0
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 : ['Z99']
你好: ['S1.1.1', 'Z4', 'Z8mf']
，: ['PUNCT']
同志们: ['S4c', 'B4']
，: ['PUNCT']
我: ['Z8mf']
想: ['X2.1']
和: ['Z5']
你们: ['Z8mf']
分享: ['S1.1.2+', 'A9+', 'F1']
我: ['Z8mf']
在: ['Z5']
手术: ['A1.1.1', 'B3', 'G3', 'B3/H1', 'B3/H2', 'S8+/T1.3', 'K4/H1', 'K4', 'M7']
后: ['M6-', 'T4--', 'T1.1.1', 'N4-', 'Z5']
对抗: ['A1.1.1', 'S1.1.1', 'E2-', 'E3-', 'M6', 'X2.1', 'B5', 'S7.1+', 'Q4.3', 'S1.2.5+', 'S7.3+', 'S8-', 'S8-/E2-', 'A6.1-']
癌症: ['B2-']
的: ['Z5']
经历: ['I3.1', 'T1.1.1', 'A2.2', 'M6', 'S1.1.1', 'P1', 'A11.1-', 'A1.8-', 'P1/T1.1.1', 'Q2.1', 'B2', 'X2.2+', '

In [ ]:
# Output tags text 1
text1_tags

{0: {' ': ['Z99'],
  '你好': ['S1.1.1', 'Z4', 'Z8mf'],
  '，': ['PUNCT'],
  '同志们': ['S4c', 'B4'],
  '我': ['Z8mf'],
  '想': ['X2.1'],
  '和': ['Z5'],
  '你们': ['Z8mf'],
  '分享': ['S1.1.2+', 'A9+', 'F1'],
  '在': ['Z5'],
  '手术': ['A1.1.1',
   'B3',
   'G3',
   'B3/H1',
   'B3/H2',
   'S8+/T1.3',
   'K4/H1',
   'K4',
   'M7'],
  '后': ['M6-', 'T4--', 'T1.1.1', 'N4-', 'Z5'],
  '对抗': ['A1.1.1',
   'S1.1.1',
   'E2-',
   'E3-',
   'M6',
   'X2.1',
   'B5',
   'S7.1+',
   'Q4.3',
   'S1.2.5+',
   'S7.3+',
   'S8-',
   'S8-/E2-',
   'A6.1-'],
  '癌症': ['B2-'],
  '的': ['Z5'],
  '经历': ['I3.1',
   'T1.1.1',
   'A2.2',
   'M6',
   'S1.1.1',
   'P1',
   'A11.1-',
   'A1.8-',
   'P1/T1.1.1',
   'Q2.1',
   'B2',
   'X2.2+',
   'X9.1+',
   'I3.2',
   'A2.1+/A3+',
   'X3'],
  '又': ['N6+', 'Z4'],
  '踢进': ['Z99'],
  '一球': ['N1'],
  '将': ['T1.1.3', 'K5.2'],
  '比分': ['K5', 'A5.1', 'N3.1', 'O2', 'K2', 'X4.1', 'N5%', 'X9.2+', 'A1.1.1'],
  '改写': ['Q3'],
  '为': ['A3+', 'Z5'],
  '3': ['N1'],
  '比': ['Z5'],
  '0': ['N1'],


---

### **STEP 5: Extracting emotion words using USAS emotion tags:** *usas_emotion_tags function*

>>**input:** piece of text

>>**output:** python dictionary where *keys* represent a sentence in the input text and *values* represent a dictionary of usas tagged emotion words and tags for that sentence

---



In [ ]:
usas_emo_cls_dict = {'E1': 'Emotional Actions, States And Processes General',
 'E1+': 'Emotional',
 'E1-': 'Unemotional',
 'E2': 'Liking',
 'E2+': 'Like',
 'E2-': 'Dislike',
 'E3': 'Calm/Violent/Angry',
 'E3+': 'Calm',
 'E3-': 'Violent/Angry',
 'E4': 'Happiness and Contentment',
 'E4.1': 'Happy/sad',
 'E4.1+': 'Happy',
 'E4.1-': 'Sad',
 'E4.2': 'Contentment',
 'E4.2+': 'Content',
 'E4.2-': 'Discontent',
 'E5': 'Bravery and Fear',
 'E5+': 'Bravery',
 'E5-': 'Fear/shock',
 'E6': 'Worry and confidence',
 'E6+': 'Confident',
 'E6-': 'Worry'}

In [ ]:
# Defining the emotion tag extraction function
def usas_emotion_tags(text):
  usas_eval_tags = {}
  sentences = re.split(r'(。|！|\!|？|\?)', text)
  for i in range(len(sentences)):
    if(sentences[i]):
      usas_list = {}
      sent = nlp(sentences[i])
      print("\n")
      print(Fore.MAGENTA + Style.BRIGHT + "--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
      print(Fore.BLUE + Style.BRIGHT + f"Sentence {i}:", sentences[i])
      for token in sent:
        tags = token._.pymusas_tags
        emo_tags = [x for x in tags if x.startswith('E')]
        if emo_tags:
          usas_list[token.text] = emo_tags
          print(Fore.MAGENTA + f'{token.text}:' + Fore.GREEN, emo_tags)
      usas_eval_tags[sentences[i]] = usas_list
  return usas_eval_tags

In [ ]:
# Text 1 emotion tag extraction
text1_emo_tags = usas_emotion_tags(text_1)



--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sentence 0:  你好，同志们，我想和你们分享我在手术后对抗癌症的经历，我又踢进一球，将比分改写为3比0
对抗: ['E2-', 'E3-']


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sentence 1: 。


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sentence 2: 希望这个故事能够激励某个人
希望: ['E4.1+']
激励: ['E3-', 'E4.1+', 'E1']


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sentenc

In [ ]:
text1_emo_tags

{' 你好，同志们，我想和你们分享我在手术后对抗癌症的经历，我又踢进一球，将比分改写为3比0': {'对抗': ['E2-', 'E3-']},
 '。': {},
 '希望这个故事能够激励某个人': {'希望': ['E4.1+'], '激励': ['E3-', 'E4.1+', 'E1']},
 '\n\n我在2020年5月被确诊为结肠癌，当时因为大出血而入院': {},
 '一眨眼之间，医生已经为我切除了30厘米的结肠': {},
 '恢复过程虽然辛苦但还能应付': {'辛苦': ['E2-', 'E4.1-', 'E6-', 'E3-'],
  '但': ['E3+/A2.1', 'E6+', 'E3+']},
 '好在虽然肿瘤很大，淋巴结却是清晰的，我被诊断为二期癌症': {'期': ['E5-']},
 '按照医生说的，我已经完全清除癌症，算是比较幸运的人之一，身后没有癌症，只剩一段可在酒吧里说说的好故事': {'清除': ['E3/L1-'],
  '幸运': ['E4.1+']},
 '那时，比分是我1-0癌症': {},
 '\n\n然而，后来我痛苦地发现，癌症也可以经血液转移': {'痛苦': ['E4.1-'], '地': ['E4.1-'], '转移': ['E1']},
 '2021年1月，我的例行检查显示出了不好的消息，我的肝脏里有好几处病灶，当外科医生告诉我肝转移无法手术时，我的世界崩塌了': {'显示': ['E2+'],
  '转移': ['E1']},
 '我现在是四期结肠癌，伴有肝转移，无法手术': {'期': ['E5-']},
 '而我还有两个小儿子，一个3岁，一个5岁，我无法让他们没了爸爸': {},
 '\n\n这一切都发生在新冠封锁时期，家人和孩子因为医院的限制，没办法随时陪伴我': {'时期': ['E5-'], '限制': ['E1-/E3+']},
 '\n\n然而，我并不想放弃，我把它看作是我需要面对的新挑战': {'放弃': ['E1%'], '挑战': ['E3-', 'E5+']},
 '一开始胜算不大，但经过长达一年半的化疗和放疗，我身上的癌症彻底被击退，我甚至不需要接受肝脏手术': {'但': ['E3+/A2.1',
   'E6+',
   'E3+'],
  '达': ['E4.1-'],
  


---

### **STEP 6: USAS Sentiment Classification:** *usas_sentiment_classifier function*

>>**input:** dictionary of sentence and emotion word tags

>>**output:** sentiment class *positive*, *negative*, *neutral* with counts per class

---



In [ ]:
# Defining the sentiment classifier
def usas_sentiment_classifier(usas_emo_tag, positive, negative, neutral):
  sentiment_cls = {}
  for sent, emo_dict in usas_emo_tag.items():
    neg, pos, neu = 0, 0, 0
    if(emo_dict):
      for key, val_list in emo_dict.items():
        for val in val_list:
          if val.endswith('-'):
            neg+=1
            negative+=1
          elif  val.endswith('+'):
            pos+=1
            positive+=1
          else:
            neu+=1
            neutral+=1
    else:
      neu+=1
      neutral+=1
    if neg==max(pos, neg, neu):
          sentiment = 'negative'
          sentiment_cls[sent] = sentiment
    elif pos==max(pos, neg, neu):
          sentiment = 'positive'
          sentiment_cls[sent] = sentiment
    else:
          sentiment = 'neutral'
          sentiment_cls[sent] = sentiment
    print(Fore.MAGENTA + Style.BRIGHT + "Sentence:", sent)
    print(Fore.GREEN + Style.BRIGHT + "class:", sentiment)
    print("\n")
    print(Fore.GREEN + Style.BRIGHT + "--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
  return sentiment_cls, positive, negative, neutral

In [ ]:
# Sentiment counts and tagging
sentiments_text1, pos_text1, neg_text1, neu_text1 = usas_sentiment_classifier(text1_emo_tags, 0, 0, 0)

Sentence:  你好，同志们，我想和你们分享我在手术后对抗癌症的经历，我又踢进一球，将比分改写为3比0
class: negative


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sentence: 。
class: neutral


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sentence: 希望这个故事能够激励某个人
class: positive


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sentence: 

我在2020年5月被确诊为结肠癌，当时因为大出血而入院
class: neutral


----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Text 1 sentiment tags per sentence
sentiments_text1

{' 你好，同志们，我想和你们分享我在手术后对抗癌症的经历，我又踢进一球，将比分改写为3比0': 'negative',
 '。': 'neutral',
 '希望这个故事能够激励某个人': 'positive',
 '\n\n我在2020年5月被确诊为结肠癌，当时因为大出血而入院': 'neutral',
 '一眨眼之间，医生已经为我切除了30厘米的结肠': 'neutral',
 '恢复过程虽然辛苦但还能应付': 'negative',
 '好在虽然肿瘤很大，淋巴结却是清晰的，我被诊断为二期癌症': 'negative',
 '按照医生说的，我已经完全清除癌症，算是比较幸运的人之一，身后没有癌症，只剩一段可在酒吧里说说的好故事': 'negative',
 '那时，比分是我1-0癌症': 'neutral',
 '\n\n然而，后来我痛苦地发现，癌症也可以经血液转移': 'negative',
 '2021年1月，我的例行检查显示出了不好的消息，我的肝脏里有好几处病灶，当外科医生告诉我肝转移无法手术时，我的世界崩塌了': 'positive',
 '我现在是四期结肠癌，伴有肝转移，无法手术': 'negative',
 '而我还有两个小儿子，一个3岁，一个5岁，我无法让他们没了爸爸': 'neutral',
 '\n\n这一切都发生在新冠封锁时期，家人和孩子因为医院的限制，没办法随时陪伴我': 'negative',
 '\n\n然而，我并不想放弃，我把它看作是我需要面对的新挑战': 'negative',
 '一开始胜算不大，但经过长达一年半的化疗和放疗，我身上的癌症彻底被击退，我甚至不需要接受肝脏手术': 'negative',
 '现在比分是2-0，我和家人都在默默祈祷癌症永远不会再回来': 'neutral',
 '\n\n它确实没有再回来，但完全不同的一场挑战又向我走来': 'positive',
 '医生在常规检查时怀疑我甲状腺出了问题，经过额外检查和穿刺活检，我被确诊为甲状腺癌': 'neutral',
 '我当时只说了一句：“呃，这次算小菜一碟': 'neutral',
 '”\n\n昨天，我进行了甲状腺全切手术，肿瘤已经有2厘米大': 'neutral',
 '\n\n现在是3-0': 'neutral',
 '我不知道接下来会面对什么，但无论

In [ ]:
# Text 1 sentiment counts for per class
print(f'Text 1: positive = {pos_text1}, negative = {neg_text1}, neutral = {neu_text1}')

Text 1: positive = 19, negative = 20, neutral = 20


In [ ]:
text1_emo_tags

{' 你好，同志们，我想和你们分享我在手术后对抗癌症的经历，我又踢进一球，将比分改写为3比0': {'对抗': ['E2-', 'E3-']},
 '。': {},
 '希望这个故事能够激励某个人': {'希望': ['E4.1+'], '激励': ['E3-', 'E4.1+', 'E1']},
 '\n\n我在2020年5月被确诊为结肠癌，当时因为大出血而入院': {},
 '一眨眼之间，医生已经为我切除了30厘米的结肠': {},
 '恢复过程虽然辛苦但还能应付': {'辛苦': ['E2-', 'E4.1-', 'E6-', 'E3-'],
  '但': ['E3+/A2.1', 'E6+', 'E3+']},
 '好在虽然肿瘤很大，淋巴结却是清晰的，我被诊断为二期癌症': {'期': ['E5-']},
 '按照医生说的，我已经完全清除癌症，算是比较幸运的人之一，身后没有癌症，只剩一段可在酒吧里说说的好故事': {'清除': ['E3/L1-'],
  '幸运': ['E4.1+']},
 '那时，比分是我1-0癌症': {},
 '\n\n然而，后来我痛苦地发现，癌症也可以经血液转移': {'痛苦': ['E4.1-'], '地': ['E4.1-'], '转移': ['E1']},
 '2021年1月，我的例行检查显示出了不好的消息，我的肝脏里有好几处病灶，当外科医生告诉我肝转移无法手术时，我的世界崩塌了': {'显示': ['E2+'],
  '转移': ['E1']},
 '我现在是四期结肠癌，伴有肝转移，无法手术': {'期': ['E5-']},
 '而我还有两个小儿子，一个3岁，一个5岁，我无法让他们没了爸爸': {},
 '\n\n这一切都发生在新冠封锁时期，家人和孩子因为医院的限制，没办法随时陪伴我': {'时期': ['E5-'], '限制': ['E1-/E3+']},
 '\n\n然而，我并不想放弃，我把它看作是我需要面对的新挑战': {'放弃': ['E1%'], '挑战': ['E3-', 'E5+']},
 '一开始胜算不大，但经过长达一年半的化疗和放疗，我身上的癌症彻底被击退，我甚至不需要接受肝脏手术': {'但': ['E3+/A2.1',
   'E6+',
   'E3+'],
  '达': ['E4.1-'],
  

---
### **STEP 7: Sentiment Probability - Sentence-level**
---



In [ ]:
# Defining the probability estimation function for sentiment classification at the sentence-level
def sentence_sentiment_prob(usas_emo_tag):
  sentiment_cls = {}
  for sent, emo_dict in usas_emo_tag.items():
    neg, pos, neu = 0, 0, 0
    if(emo_dict):
      for key, val_list in emo_dict.items():
        for val in val_list:
          if val.endswith('-'):
            neg+=1
          elif val.endswith('+'):
            pos+=1
          else:
            neu+=1
    else:
      neu+=1
    sentiment = f'positive={pos/(pos+neg+neu)}, negative={neg/(pos+neg+neu)}, neutral={neu/(pos+neg+neu)}'
    sentiment_cls[sent] = sentiment
    print(Fore.GREEN + Style.BRIGHT + "Sentence:", sent)
    print(Fore.CYAN + Style.BRIGHT + "class:", sentiment)
    print("\n")
    print(Fore.CYAN + Style.BRIGHT + "--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
  return sentiment_cls

In [ ]:
# Text 1 sentence-level sentiment probability
sentiment_prob_text1 = (sentence_sentiment_prob(text1_emo_tags))

Sentence:  你好，同志们，我想和你们分享我在手术后对抗癌症的经历，我又踢进一球，将比分改写为3比0
class: positive=0.0, negative=1.0, neutral=0.0


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sentence: 。
class: positive=0.0, negative=0.0, neutral=1.0


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sentence: 希望这个故事能够激励某个人
class: positive=0.5, negative=0.25, neutral=0.25


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Sentence: 

我在2020年5月被确诊为结肠癌，当时因为大出血而入院
class: positive=0.0, negative=0.0, neutral=1.0


--------------------------------------------------

In [ ]:
sentiment_prob_text1

{' 你好，同志们，我想和你们分享我在手术后对抗癌症的经历，我又踢进一球，将比分改写为3比0': 'positive=0.0, negative=1.0, neutral=0.0',
 '。': 'positive=0.0, negative=0.0, neutral=1.0',
 '希望这个故事能够激励某个人': 'positive=0.5, negative=0.25, neutral=0.25',
 '\n\n我在2020年5月被确诊为结肠癌，当时因为大出血而入院': 'positive=0.0, negative=0.0, neutral=1.0',
 '一眨眼之间，医生已经为我切除了30厘米的结肠': 'positive=0.0, negative=0.0, neutral=1.0',
 '恢复过程虽然辛苦但还能应付': 'positive=0.2857142857142857, negative=0.5714285714285714, neutral=0.14285714285714285',
 '好在虽然肿瘤很大，淋巴结却是清晰的，我被诊断为二期癌症': 'positive=0.0, negative=1.0, neutral=0.0',
 '按照医生说的，我已经完全清除癌症，算是比较幸运的人之一，身后没有癌症，只剩一段可在酒吧里说说的好故事': 'positive=0.5, negative=0.5, neutral=0.0',
 '那时，比分是我1-0癌症': 'positive=0.0, negative=0.0, neutral=1.0',
 '\n\n然而，后来我痛苦地发现，癌症也可以经血液转移': 'positive=0.0, negative=0.6666666666666666, neutral=0.3333333333333333',
 '2021年1月，我的例行检查显示出了不好的消息，我的肝脏里有好几处病灶，当外科医生告诉我肝转移无法手术时，我的世界崩塌了': 'positive=0.5, negative=0.0, neutral=0.5',
 '我现在是四期结肠癌，伴有肝转移，无法手术': 'positive=0.0, negative=1.0, neutral=0.0',
 '而我还有两个小儿子，一个3岁，一个5岁，我无法让他

---
### **STEP 8: Sentiment Probability -Text-level**
---



In [ ]:
def text_sentiment_prob(text, pos, neg, neu):
  prob = f'positive={pos/(pos+neg+neu)}, negative={neg/(pos+neg+neu)}, neutral={neu/(pos+neg+neu)}'
  print(Fore.CYAN+ text)
  print(Fore.GREEN+ "----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
  print(prob)
  return prob

In [ ]:
text1_prob = text_sentiment_prob(text_1, pos_text1, neg_text1, neu_text1)

 你好，同志们，我想和你们分享我在手术后对抗癌症的经历，我又踢进一球，将比分改写为3比0。希望这个故事能够激励某个人。

我在2020年5月被确诊为结肠癌，当时因为大出血而入院。一眨眼之间，医生已经为我切除了30厘米的结肠。恢复过程虽然辛苦但还能应付。好在虽然肿瘤很大，淋巴结却是清晰的，我被诊断为二期癌症。按照医生说的，我已经完全清除癌症，算是比较幸运的人之一，身后没有癌症，只剩一段可在酒吧里说说的好故事。那时，比分是我1-0癌症。

然而，后来我痛苦地发现，癌症也可以经血液转移。2021年1月，我的例行检查显示出了不好的消息，我的肝脏里有好几处病灶，当外科医生告诉我肝转移无法手术时，我的世界崩塌了。我现在是四期结肠癌，伴有肝转移，无法手术。而我还有两个小儿子，一个3岁，一个5岁，我无法让他们没了爸爸。

这一切都发生在新冠封锁时期，家人和孩子因为医院的限制，没办法随时陪伴我。

然而，我并不想放弃，我把它看作是我需要面对的新挑战。一开始胜算不大，但经过长达一年半的化疗和放疗，我身上的癌症彻底被击退，我甚至不需要接受肝脏手术。现在比分是2-0，我和家人都在默默祈祷癌症永远不会再回来。

它确实没有再回来，但完全不同的一场挑战又向我走来。医生在常规检查时怀疑我甲状腺出了问题，经过额外检查和穿刺活检，我被确诊为甲状腺癌。我当时只说了一句：“呃，这次算小菜一碟。”

昨天，我进行了甲状腺全切手术，肿瘤已经有2厘米大。

现在是3-0。我不知道接下来会面对什么，但无论发生什么，我都会和这个混蛋斗争到底。现在我每6个月都会进行预防性的化疗，并继续享受和孩子们共度的时光。

所以，同志，面对你的挑战吧。不管生活有多么艰难，你也有力量和乐观坚持下去。昂起头，继续进球！
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
positive=0.3220338983050847, negative=0.33898